In [1]:
import pandas as pd

In [2]:
#HELPER FUNCTIONS
def remove_number(string, cell_types):
    for x in cell_types:
        if string[:len(x)] == x:
            return(x)

        
def try_float(v):
    try:
       return float(v)
    except Exception:
       return 0



In [32]:
#CONSTANTS

filename = "mc38b_ps"
gene_index = 0

In [33]:
#LOAD FILES

#scrna data
x = []
file = open("mc38b_aggexp_counts.csv", "r")
for l in file:
    x.append(l.split(","))
print(x[2][:5])

#mass spec file
ms = pd.read_excel("input_data/deconvolution_parsed/ps_parsed.xlsx")
print(ms.head)

#sample key
samples = pd.read_excel("input_data/deconvolution_parsed/sample_key.xlsx")
print(samples.head)

['"Dendritic"', '0', '1', '3', '1']
<bound method NDFrame.head of      gene_symbol  Site Position          Motif  1022 SET2 1-2  668 set2 2-1  \
0           Dhx9            137  ESSGYGSPGPTWD      19.434117     20.969235   
1          Psma3            250  LKEEDESDDDNMx     105.537861    115.251424   
2           Tns1           1547  PEKRRMSVGDRAG      29.428897     22.230125   
3           Tns1            604  TAPAALSPQEKKE       0.000000      0.000000   
4           Tns1            836  PRQQERSPLQSLA      11.349067     10.068697   
...          ...            ...            ...            ...           ...   
8603        Tap1            664  QRLLYESPKRASR       0.000000      0.000000   
8604        Gbp2            216  ADKKTKSFNEPRL      10.702340     12.063204   
8605       Rps28             23  VLGRTGSQGQCTQ       0.000000      0.000000   
8606       Stx12            142  RAGSRLSAEDRQR     107.871926    101.532068   
8607       Stx12            139  ARARAGSRLSAED      30.550351    

In [34]:
samples = samples.values.tolist()
sample_map = {}
for i in range(len(samples)):
    sample_map[samples[i][0]] = samples[i][1]

In [35]:
samples = []
sample_dict = {}
for elt in x[1:]:
    samples.append(elt[-1].replace('"', '').strip())
    sample_dict[elt[-1].replace('"', '').strip()] = {}

In [36]:
cell_types = set()
for elt in x:
    if x[1][-1] == elt[-1]:
        cell_types.add(elt[0].replace('"', '').strip())
        


In [37]:
#DETERMINE WHICH ROWS ARE MISSING
#make set of sample_celltype strings in rna aggexp (remember to strip numbers)

rna_samplecelltype = [tuple([remove_number(elt[0].replace('"', '').strip(), cell_types),elt[-1].replace('"', '').strip()])for elt in x[1:]]

#make all possible sample_celltype strings and subtract sets
possible_samplecelltype = []
for i in cell_types:
    for j in set(samples):
        possible_samplecelltype.append(tuple([i,j]))
        
        
missing = set(possible_samplecelltype)-set(rna_samplecelltype)

#add back a row of 1s of the proper length
new = []
for i in missing:
    row = [i[0]] + ["1"]*(len(x[0])-2) + [i[1]]
    new.append(row)

x.extend(new)



{('fibroblasts', '4_3'), ('endothelial cells', '1_2'), ('endothelial cells', '4_5'), ('fibroblasts', '3_5')}


In [39]:
x2 = []
for i in x:
    new = list(map(lambda x: x.replace('0', '1'), i))
    x2.append(new)
x = x2

In [40]:

genes = [z.replace('"', '').strip() for z in x[0][:-1]]
df = x[1:]
for elt in df:
    sample = elt[-1].replace('"', '').strip()
    for elt2 in range(1, len(genes)-2):
        if genes[elt2] not in sample_dict[sample].keys():
            sample_dict[sample][genes[elt2]] = [0, 0, ""]
        current = sample_dict[sample][genes[elt2]]
        value = try_float(elt[elt2])
        if value > current[0]:
            sample_dict[sample][genes[elt2]][0] = value
            sample_dict[sample][genes[elt2]][2] = elt[0].replace('"', '').strip()
        sample_dict[sample][genes[elt2]][1] += value


In [41]:

all_dict = {}
for elt in x[1:]:
    all_dict[elt[-1].replace('"', '').strip()] = {}
    
for elt in x[1:]:
    all_dict[elt[-1].replace('"', '').strip()][remove_number(elt[0].replace('"', '').strip(), cell_types)] = {}
    for elt2 in range(1, len(genes)):
        all_dict[elt[-1].replace('"', '').strip()][remove_number(elt[0].replace('"', '').strip(), cell_types)][genes[elt2]] = elt[elt2]

print(all_dict.keys())

dict_keys(['3_4', '2_1', '2_4', '4_1', '3_5', '1_1', '3_1', '1_4', '1_2', 'background', '4_3', '4_5', '3_2'])


In [42]:
out = []

prot = ms.values.tolist()
prot_cols = ms.columns

for elt in prot:
    if elt[gene_index] not in genes:  
        continue
    row = []
    for elt2 in range(len(elt)):
        
        if prot_cols[elt2] not in sample_map.keys():
            row.append(elt[elt2])
        else:
            sample_id = sample_map[prot_cols[elt2]]
            total_signal = sample_dict[sample_id][elt[gene_index]][1]
            for elt3 in cell_types:
                if elt3 in all_dict[sample_id].keys():
                    specific_signal = all_dict[sample_id][elt3][elt[gene_index]]
                else:
                    specific_signal = 0
                if total_signal == 0:
                    row.append("NA")
                else:
                    row.append(float(elt[elt2])*float(specific_signal)/total_signal)
    out.append(row)

In [43]:
out_cols = []
for elt in prot:
    if elt[gene_index] not in genes:
        continue
    row = []
    for elt2 in range(len(elt)):
        if prot_cols[elt2] not in sample_map.keys():
            out_cols.append(prot_cols[elt2])
        else:
            sample_id = prot_cols[elt2]
            for elt3 in cell_types:
                out_cols.append(sample_id + "_" + elt3)
                
    break


In [44]:
out_df = pd.DataFrame(out, columns = out_cols)
out_df.head()

,gene_symbol,Site Position,Motif,1022 SET2 1-2_T/NK cells,1022 SET2 1-2_Dendritic,1022 SET2 1-2_EMT,1022 SET2 1-2_Cancer cells,1022 SET2 1-2_endothelial cells,1022 SET2 1-2_neutrophils,1022 SET2 1-2_fibroblasts,...,495 set2 3-5_fibroblasts,495 set2 3-5_Myeloid,382 set2 1-4_T/NK cells,382 set2 1-4_Dendritic,382 set2 1-4_EMT,382 set2 1-4_Cancer cells,382 set2 1-4_endothelial cells,382 set2 1-4_neutrophils,382 set2 1-4_fibroblasts,382 set2 1-4_Myeloid
0,Dhx9,137,ESSGYGSPGPTWD,0.391291,0.130430,12.130019,5.608503,0.130430,0.130430,0.130430,...,0.107210,0.857680,0.050293,0.050293,7.242206,7.845724,0.050293,0.050293,0.050293,0.301759
1,Psma3,250,LKEEDESDDDNMx,1.554959,0.598061,61.919266,34.966643,0.039871,0.239224,0.438578,...,0.028145,7.936942,0.441588,0.315420,47.249896,48.700827,0.294392,0.021028,0.862148,3.658870
2,Tns1,1547,PEKRRMSVGDRAG,3.269877,3.269877,3.269877,6.539755,3.269877,3.269877,3.269877,...,1.198085,3.594254,1.662704,1.662704,6.650814,3.325407,1.662704,1.662704,6.650814,4.988111
3,Tns1,604,TAPAALSPQEKKE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Tns1,836,PRQQERSPLQSLA,1.261007,1.261007,1.261007,2.522015,1.261007,1.261007,1.261007,...,0.300321,0.900962,0.620469,0.620469,2.481876,1.240938,0.620469,0.620469,2.481876,1.861407


In [45]:
out_df.to_csv(filename + ".csv")